# Toy Problem - Balances and Uncertainty 

Adapted from multiple AdvLCA scripts. Creating a toy problem from ecoinvent process to demonstrate the effect of uncertainty analysis on mass balances

From Adv LCA Course Notes:

This script shows how to add uncertainties to your home-made product system and run a Monte Carlo simulation. 

To better understand this script, I recommend reading read the paper by Limpert et al. (2001), that is a 'classic' and explains the log-normal distribution.

_Limpert, E., Stahel, W. A., & Abbt, M. (2001). Log-normal distributions across the sciences: Keys and clues. Bioscience, 51(5), 341-352. [https://doi.org/10.1641/0006-3568(2001)051[0341:LNDATS]2.0.CO;2](https://academic.oup.com/bioscience/article/51/5/341/243981)_

In [176]:
import brightway2 as bw
import bw2data
import bw2analyzer
import pandas as pd
import numpy as np

from lci_to_bw2 import * # import all the functions of this module

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

from scipy import stats



import openpyxl

In [177]:
mydb = pd.read_csv('balancedb_excel_w_ecoinvent.csv', header = 0, sep = ",") # using csv file avoids encoding problem
mydb.head()

,Activity database,Activity code,Activity name,Activity unit,Activity type,Exchange database,Exchange input,Exchange amount,Exchange unit,Exchange type,...,Exchange negative,Notes,Variance - Basic,Variance - Pedigree,Variance - Total,"CI/2wP, half range of confidence interval",sigma,sigma*,sigma* - basic,sigma* - relative
0,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Concrete production,2316.0,kilogram,production,...,NaN,NaN,NaN,NaN,0.0000,1.000000,1.000000,0.000000,NaN,NaN
1,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Cement,176.0,kilogram,technosphere,...,NaN,NaN,0.04,0.0127,0.0527,1.582696,1.258052,0.229565,0.2,0.017304
2,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Water,176.0,kilogram,technosphere,...,NaN,NaN,0.04,0.0127,0.0527,1.582696,1.258052,0.229565,0.2,0.017304
3,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Fine Aggregate,725.0,kilogram,technosphere,...,NaN,NaN,0.04,0.0127,0.0527,1.582696,1.258052,0.229565,0.2,0.008526
4,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Course Aggregate,1169.0,kilogram,technosphere,...,NaN,NaN,0.04,0.0127,0.0527,1.582696,1.258052,0.229565,0.2,0.006714


In [178]:
# clean up a bit
mydb = mydb.drop('Notes', 1)  # remove the columns not needed
mydb['Exchange uncertainty type'] = mydb['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
# Note: to avoid having both nan and values in the uncertainty column I use zero as default
mydb.head()

C:\Users\em398\AppData\Local\Temp\ipykernel_20168\3480241213.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  mydb = mydb.drop('Notes', 1)  # remove the columns not needed


,Activity database,Activity code,Activity name,Activity unit,Activity type,Exchange database,Exchange input,Exchange amount,Exchange unit,Exchange type,...,Exchange scale,Exchange negative,Variance - Basic,Variance - Pedigree,Variance - Total,"CI/2wP, half range of confidence interval",sigma,sigma*,sigma* - basic,sigma* - relative
0,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Concrete production,2316.0,kilogram,production,...,NaN,NaN,NaN,NaN,0.0000,1.000000,1.000000,0.000000,NaN,NaN
1,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Cement,176.0,kilogram,technosphere,...,0.229565,NaN,0.04,0.0127,0.0527,1.582696,1.258052,0.229565,0.2,0.017304
2,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Water,176.0,kilogram,technosphere,...,0.229565,NaN,0.04,0.0127,0.0527,1.582696,1.258052,0.229565,0.2,0.017304
3,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Fine Aggregate,725.0,kilogram,technosphere,...,0.229565,NaN,0.04,0.0127,0.0527,1.582696,1.258052,0.229565,0.2,0.008526
4,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Course Aggregate,1169.0,kilogram,technosphere,...,0.229565,NaN,0.04,0.0127,0.0527,1.582696,1.258052,0.229565,0.2,0.006714


In [179]:
#Choose test - Basic (0.02) + Additional 
# 0 - [0,0,0,0,0] NOT USED
# 1 - [1,1,1,1,1] data quality score (0)
# 2 - [2,2,2,2,2] data quality score (0.001525)
# 3 - [3,3,3,3,3] data quality score (0.0127)
# 4 - [4,4,4,4,4] data quality score (0.0586)
# 5 - [5,5,5,5,5] data quality score (0.21)

uncertainty_Char = 3


exc_scales = np.array([
    [0,0,0,0,0,0],
    [0.141421356,0.141421356,0.141421356,0.141421356,0.141421356,0],
    [0.146714008, 0.146714008,0.146714008,0.146714008,0.146714008,0],
    [0.180831413, 0.180831413, 0.180831413, 0.180831413, 0.180831413,0],
    [0.280356915, 0.280356915, 0.280356915, 0.280356915, 0.280356915, 0],
    [0.479583152, 0.479583152, 0.479583152, 0.479583152, 0.479583152, 0]
])

exc_scale = exc_scales[uncertainty_Char]
exc_scale

array([0.18083141, 0.18083141, 0.18083141, 0.18083141, 0.18083141,
       0.        ])

In [180]:
bw.projects.set_current('Balances_Cement')

In [181]:
bw.bw2setup()

Biosphere database already present!!! No setup is needed


In [182]:
#bw.databases.clear() # line to use in case you had already databases in the project space
bw.databases # lists all databases. We start from an empty project

Databases dictionary with 2 object(s):
	balancedb_extended
	biosphere3

In [183]:
# This cell is to clean up
#del bw.databases['balancedb'] 


In [184]:
# Create a dict that can be written as database
bw2_db = lci_to_bw2(mydb) # a function from the lci_to_bw2 module
bw2_db

{('balancedb_extended', 'Concrete production'): {'name': 'Concrete production',
  'unit': 'kilogram',
  'type': 'process',
  'exchanges': [{'input': ('balancedb_extended', 'Concrete production'),
    'amount': 2316.0,
    'unit': 'kilogram',
    'type': 'production',
    'uncertainty type': 0,
    'Variance - Total': 0.0,
    'CI/2wP, half range of confidence interval': 1.0,
    'sigma': 1.0,
    'sigma*': 0.0},
   {'input': ('balancedb_extended', 'Cement'),
    'amount': 176.0,
    'unit': 'kilogram',
    'type': 'technosphere',
    'uncertainty type': 2,
    'loc': 5.170483995,
    'scale': 0.229564806,
    'Variance - Basic': 0.04,
    'Variance - Pedigree': 0.0127,
    'Variance - Total': 0.0527,
    'CI/2wP, half range of confidence interval': 1.582695825,
    'sigma': 1.258052394,
    'sigma*': 0.229564806,
    'sigma* - basic': 0.2,
    'sigma* - relative': 0.017304098},
   {'input': ('balancedb_extended', 'Water'),
    'amount': 176.0,
    'unit': 'kilogram',
    'type': 'techn

In [185]:
t_db = bw.Database('balancedb_extended') # it works because the database name in the excel file is the same
# shut down all other notebooks using the same project
t_db.write(bw2_db)

Writing activities to SQLite3 database:
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/15/2024 11:01:00
  Finished: 01/15/2024 11:01:00
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 1.55


A short recap from yesterday, let's create a database and run LCA calcualtions

In [186]:

functional_unit = {t_db.get('Concrete production'): 2316}
lca = bw.LCA(functional_unit)  # LCA calculations with method
lca.lci()
print(lca.inventory) # Is this what you expected? #equivalent to environmental vector from the LCI-matrix. 

  (0, 5)	2.0299999415874486
  (0, 4)	3.5070000304840514
  (0, 3)	1.5224999457132073
  (0, 2)	0.18034720048308375
  (0, 1)	168.6080007553101


In [187]:
#Test with other biosphere flow having a characterisation factor. (Compared to previous 1 for carbon dioxide and 0 for ethylene oxide)
#Compare to deterministic score (relative diff for both biosphere flows)


myLCIAdata = [[('balancedb_extended', 'Carbon dioxide'), 1.0]
]
method_key = ('simplemethod', 'imaginaryendpoint', 'imaginarymidpoint')
my_method = bw.Method(method_key)
my_method.validate(myLCIAdata)
my_method.register()
my_method.write(myLCIAdata)
my_method.load()


[[('balancedb_extended', 'Carbon dioxide'), 1.0]]

In [188]:
lca = bw.LCA(functional_unit, method_key)
lca.lci(factorize=True)
lca.lcia()
lca_df = lca.to_dataframe()
print('characterised_inventory\n', lca.characterized_inventory.toarray())

print('Score \n',lca.score)

det_LCA = lca.score



characterised_inventory
 [[  0.         168.60800076   0.1803472    1.52249995   3.50700003
    2.02999994]]
Score 
 175.84784787357788


This is the **static** or **deterministic** result of this LCA. 

In [189]:
lca_df

,Activity,Flow,Region,Amount
0,Fly-ash,Carbon dioxide,None,168.608001
1,Course Aggregate,Carbon dioxide,None,3.507000
2,Fine Aggregate,Carbon dioxide,None,2.030000
3,Water,Carbon dioxide,None,1.522500
4,Cement,Carbon dioxide,None,0.180347


In [190]:

#pd.DataFrame(lca.technosphere_matrix.toarray().getI())

In [191]:
pd.DataFrame(lca.demand_array)

,0
0,2316.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0


In [192]:
pd.DataFrame(lca.supply_array)

,0
0,1.0
1,176.0
2,176.0
3,725.0
4,1169.0
5,70.0


# Characterise uncertainty 


**Note**: uncertainties are always added to EXCHANGES (not to activities...)


In [193]:
ec = t_db.get('Concrete production')  


In [194]:
exc = list(ec.exchanges())
exc

[Exchange: 2316.0 kilogram 'Concrete production' (kilogram, None, None) to 'Concrete production' (kilogram, None, None)>,
 Exchange: 176.0 kilogram 'Cement' (kilogram, None, None) to 'Concrete production' (kilogram, None, None)>,
 Exchange: 176.0 kilogram 'Water' (kilogram, None, None) to 'Concrete production' (kilogram, None, None)>,
 Exchange: 725.0 kilogram 'Fine Aggregate' (kilogram, None, None) to 'Concrete production' (kilogram, None, None)>,
 Exchange: 1169.0 kilogram 'Course Aggregate' (kilogram, None, None) to 'Concrete production' (kilogram, None, None)>,
 Exchange: 70.0 kilogram 'Fly-ash' (kilogram, None, None) to 'Concrete production' (kilogram, None, None)>]

In [195]:
#ec.as_dict()

In [196]:
# Lognormal distribution first
from stats_arrays import LognormalUncertainty
import numpy as np

index = 0
for exchange in exc:
    if exchange['input'] != exchange['output']:
        exchange['uncertainty type'] = LognormalUncertainty.id # this is an integer (not a float)
        exchange['loc'], exchange['scale'] = np.log(exchange['amount']), exc_scale[index] 

        if exchange['amount']<0:
            exchange['loc'] = np.log(-exchange['amount'])
            exchange['negative'] = True
        exchange.save() # important
        print(exchange.uncertainty)
        index += 1
    
    



{'uncertainty type': 2, 'loc': 5.170483995038151, 'scale': 0.180831413}
{'uncertainty type': 2, 'loc': 5.170483995038151, 'scale': 0.180831413}
{'uncertainty type': 2, 'loc': 6.586171654854675, 'scale': 0.180831413}
{'uncertainty type': 2, 'loc': 7.063903961472068, 'scale': 0.180831413}
{'uncertainty type': 2, 'loc': 4.248495242049359, 'scale': 0.180831413}


In [197]:
exc[1].as_dict()  # Now uncertainty is included

{'input': ('balancedb_extended', 'Cement'),
 'amount': 176.0,
 'unit': 'kilogram',
 'type': 'technosphere',
 'uncertainty type': 2,
 'loc': 5.170483995038151,
 'scale': 0.180831413,
 'Variance - Basic': 0.04,
 'Variance - Pedigree': 0.0127,
 'Variance - Total': 0.0527,
 'CI/2wP, half range of confidence interval': 1.582695825,
 'sigma': 1.258052394,
 'sigma*': 0.229564806,
 'sigma* - basic': 0.2,
 'sigma* - relative': 0.017304098,
 'output': ('balancedb_extended', 'Concrete production')}

# MC simulation

From Adv LCA course notes:
Sources here: [example](http://stackoverflow.com/questions/38532146/obtaining-distribution-of-results-from-lcia), [another example](https://brightwaylca.org/examples/getting-started.html), and [source code](https://bitbucket.org/cmutel/brightway2-calc/src/662740694a8c70074105b5dca45b58651adb5eb5/bw2calc/monte_carlo.py?at=default&fileviewer=file-view-default)

In [198]:

amount = 2316
iterations = 1000
# This is the montecarlo simulation
mc = bw.MonteCarloLCA({ec: amount}, method_key)  # Monte Carlo class


Redo deterministic LCA

In [199]:
lca = bw.LCA({ec: amount}, method_key)
lca.lci()
lca.lcia()


det_bioMatrix = lca.biosphere_matrix.toarray()
det_techMatrix = lca.technosphere_matrix.toarray()


print('technosphere matrix\n',pd.DataFrame(det_techMatrix))
print('biosphere matrix\n',pd.DataFrame(det_bioMatrix))
print('supply array\n',pd.DataFrame(lca.supply_array))
print('characterised_inventory\n', pd.DataFrame(lca.characterized_inventory.toarray()))


det_LCA = lca.score
det_bioProcess = det_bioMatrix[:,0]

det_techProcess = det_techMatrix[:,0]
det_balance =  det_techProcess.sum()+ det_bioProcess.sum()

print('LCIA Score \n',lca.score)
print('Process Biosphere Column \n', det_bioProcess)

print('Process Technosphere Column \n',det_techProcess)
print('Mass Balance Percent \n', det_balance*100, ' %')


technosphere matrix
         0    1    2    3    4    5
0  2316.0  0.0  0.0  0.0  0.0  0.0
1  -176.0  1.0  0.0  0.0  0.0  0.0
2  -176.0  0.0  1.0  0.0  0.0  0.0
3  -725.0  0.0  0.0  1.0  0.0  0.0
4 -1169.0  0.0  0.0  0.0  1.0  0.0
5   -70.0  0.0  0.0  0.0  0.0  1.0
biosphere matrix
      0      1         2       3      4      5
0  0.0  0.958  0.001025  0.0021  0.003  0.029
supply array
         0
0     1.0
1   176.0
2   176.0
3   725.0
4  1169.0
5    70.0
characterised_inventory
      0           1         2       3      4     5
0  0.0  168.608001  0.180347  1.5225  3.507  2.03
LCIA Score 
 175.84784787357788
Process Biosphere Column 
 [0.]
Process Technosphere Column 
 [ 2316.  -176.  -176.  -725. -1169.   -70.]
Mass Balance Percent 
 0.0  %


In [200]:
pd.DataFrame(lca.characterized_inventory.toarray())

,0,1,2,3,4,5
0,0.0,168.608001,0.180347,1.5225,3.507,2.03


Redo Monte-Carlo

In [201]:
scores = []  # 1-dimensional array filled with zeros
diff = []  # 1-dimensional array filled with zeros
diff_bio = []
supply=[]
bio=[]
tech=[]
balance=[]
LCA_charCO2 = []
LCA_charCO2_Percent =[]
LCA_charEO = []
LCA_charEO_Percent =[]
cutOffList01 = []
cutOffList05 = []
names=['Product/Tech - Concrete production',
'Tech - Cement',
'Tech - Water',
'Tech - Fine aggregate',
'Tech - Coarse aggregate',
'Tech - Fly-ash'
]


for iteration in range(iterations):
    next(mc)
    mc_techProcess = mc.technosphere_matrix.toarray()[:,0]
    mc_bioProcess = mc.biosphere_matrix.toarray()[:,0]
    mc_LCAcharCO2 = mc.characterized_inventory.toarray()[0,:]
    mc_LCAcharCO2_Percent = (mc_LCAcharCO2 /mc.score)
    #mc_LCAcharEO = mc.characterized_inventory.toarray()[1,:]
    #mc_LCAcharEO_Percent = (mc_LCAcharEO /mc.score)
    mc_balance = mc_techProcess.sum()+mc_bioProcess.sum()
    if np.abs(mc_balance) <= 0.01:
        cutOff01 = True
    else:
        cutOff01 = False
    if np.abs(mc_balance) <= 0.05:
        cutOff05 = True
    else:
        cutOff05 = False 
        
    LCA_charCO2.append(mc_LCAcharCO2.transpose())
    LCA_charCO2_Percent.append(mc_LCAcharCO2_Percent.transpose())
    #LCA_charEO.append(mc_LCAcharEO.transpose())
    #LCA_charEO_Percent.append(mc_LCAcharEO_Percent.transpose())
    scores.append(mc.score)
    supply.append(mc.supply_array)
    bio.append(mc.biosphere_matrix.toarray()[:,0])
    tech.append(mc_techProcess)
    balance.append(mc_balance) 

    cutOffList01.append(cutOff01)
    cutOffList05.append(cutOff05)
    

In [202]:

#Names order
#bio_df = pd.DataFrame(bio,columns=names[4:6])
tech_df = pd.DataFrame(tech,columns=names[0:6])
LCA_charCO2_df = pd.DataFrame(LCA_charCO2,columns=names[0:6])
LCA_charCO2_percent_df = pd.DataFrame(LCA_charCO2_Percent,columns=names[0:6])
#LCA_charEO_df = pd.DataFrame(LCA_charEO,columns=names[0:4])
#LCA_charEO_percent_df = pd.DataFrame(LCA_charEO_Percent,columns=names[0:4])
#diff_Bio = bio_df-det_bioProcess
#diff_Bio_Rel = diff_Bio/det_bioProcess
diff_Tech = tech_df-det_techProcess
diff_Tech_Rel = diff_Tech/det_techProcess
supply_df = pd.DataFrame(supply,columns=names[0:6])
supply_transpose = supply_df.transpose()
balance_df = pd.DataFrame({"Mass Balance": balance})
MassBal_diff_Abs = balance_df.rename(columns={"Mass Balance": "Diff Mass Bal"}) - det_balance
scores_df = pd.DataFrame({"LCIA Score":scores})
diff_Score = scores_df.rename(columns={"LCIA Score": "Diff Score"}) - det_LCA


diffAbs_df = pd.concat([
    #diff_Bio, 
    diff_Tech],axis=1)
diffRel_df = pd.concat([
    #diff_Bio_Rel, 
    diff_Tech_Rel],axis=1)

diffAbs_df["Type"] = "Abs diff"
diffRel_df["Type"] = "Rel diff"
LCA_charCO2_df["Type"] = "Process CO2 LCIA absolute"
LCA_charCO2_percent_df["Type"] = "Process CO2 LCIA contribution"
LCA_charCO2_df = LCA_charCO2_df.rename_axis('Iteration')
LCA_charCO2_percent_df = LCA_charCO2_percent_df.rename_axis('Iteration')
#LCA_charEO_df["Type"] = "Process EO LCIA absolute"
#LCA_charEO_percent_df["Type"] = "Process EO LCIA contribution"
#LCA_charEO_df = LCA_charEO_df.rename_axis('Iteration')
#LCA_charEO_percent_df = LCA_charEO_percent_df.rename_axis('Iteration')
diffAbs_df = diffAbs_df.rename_axis('Iteration')
diffRel_df = diffRel_df.rename_axis('Iteration')

quantities_df = pd.concat([
tech_df
#,bio_df
],axis=1)

quantities_df = quantities_df.rename_axis('Iteration')


#balance
MCresults_df = pd.concat([
    scores_df, 
    balance_df,
    MassBal_diff_Abs,
    diff_Score
    ], axis=1)

#Cutoff_df['MB Cut Off 0.01'] = cutOffList01
#Cutoff_df['MB Cut Off 0.05'] = cutOffList05

MCresults_df = MCresults_df.rename_axis('Iteration')

quantities_df["Type"] = "Quantity (kg)"

quantities_df=pd.concat([
    quantities_df,
    diffAbs_df,
    diffRel_df,
    LCA_charCO2_df,
    LCA_charCO2_percent_df,
    #LCA_charEO_df,
    #LCA_charEO_percent_df
    ], axis=0)




MCresults_df.round(3)

,LCIA Score,Mass Balance,Diff Mass Bal,Diff Score
Iteration,,,,
0,166.664,160.059,160.059,-9.183
1,179.823,-3.488,-3.488,3.975
2,194.907,83.502,83.502,19.059
3,161.764,49.680,49.680,-14.084
4,186.143,72.456,72.456,10.295
...,...,...,...,...
995,191.322,81.374,81.374,15.474
996,277.203,138.286,138.286,101.355
997,151.892,-326.920,-326.920,-23.956


In [203]:
#MCresults_df

In [204]:
quantities_df




,Product/Tech - Concrete production,Tech - Cement,Tech - Water,Tech - Fine aggregate,Tech - Coarse aggregate,Tech - Fly-ash,Type
Iteration,,,,,,,
0,2316.0,-166.288684,-164.221364,-638.931359,-1098.407441,-88.092371,Quantity (kg)
1,2316.0,-179.965101,-216.995915,-613.351443,-1233.074278,-76.100770,Quantity (kg)
2,2316.0,-196.429936,-161.513700,-609.021729,-1208.380723,-57.152112,Quantity (kg)
3,2316.0,-160.857957,-177.907523,-710.959265,-1126.705612,-89.889749,Quantity (kg)
4,2316.0,-186.782409,-144.797088,-739.557280,-1096.000362,-76.407127,Quantity (kg)
...,...,...,...,...,...,...,...
995,0.0,0.962921,0.001356,0.007548,0.016014,0.012161,Process CO2 LCIA contribution
996,0.0,0.977156,0.000557,0.004933,0.011192,0.006161,Process CO2 LCIA contribution
997,0.0,0.941765,0.001334,0.014054,0.023239,0.019608,Process CO2 LCIA contribution


# Dataset manipulation


In [205]:
MC_results_df = quantities_df.rename_axis('Type',axis='columns')

MCresults_df_long = pd.melt(MCresults_df,
                  var_name='Type', value_name='Value',ignore_index=False)

MCresults_df_long = MCresults_df_long.reset_index(drop=False)
MCresults_df_long['Exchanges'] = "Total - Process Level" 

MCresults_df_long

,Iteration,Type,Value,Exchanges
0,0,LCIA Score,166.664494,Total - Process Level
1,1,LCIA Score,179.823107,Total - Process Level
2,2,LCIA Score,194.906882,Total - Process Level
3,3,LCIA Score,161.764160,Total - Process Level
4,4,LCIA Score,186.142800,Total - Process Level
...,...,...,...,...
3995,995,Diff Score,15.473938,Total - Process Level
3996,996,Diff Score,101.354832,Total - Process Level
3997,997,Diff Score,-23.955812,Total - Process Level
3998,998,Diff Score,-15.675135,Total - Process Level


In [206]:

#quantities_df = quantities_df.rename_axis('Exchanges',axis='columns')
#quantities_df

quantities_df_long = pd.melt(quantities_df, id_vars=['Type'],value_vars=names,
                  var_name='Exchanges', value_name='Value',ignore_index=False)

quantities_df_long = quantities_df_long.reset_index(drop=False)


quantities_df_long

,Iteration,Type,Exchanges,Value
0,0,Quantity (kg),Product/Tech - Concrete production,2316.000000
1,1,Quantity (kg),Product/Tech - Concrete production,2316.000000
2,2,Quantity (kg),Product/Tech - Concrete production,2316.000000
3,3,Quantity (kg),Product/Tech - Concrete production,2316.000000
4,4,Quantity (kg),Product/Tech - Concrete production,2316.000000
...,...,...,...,...
29995,995,Process CO2 LCIA contribution,Tech - Fly-ash,0.012161
29996,996,Process CO2 LCIA contribution,Tech - Fly-ash,0.006161
29997,997,Process CO2 LCIA contribution,Tech - Fly-ash,0.019608
29998,998,Process CO2 LCIA contribution,Tech - Fly-ash,0.011077


In [207]:
combined_df_long = pd.concat([quantities_df_long, MCresults_df_long], axis=0)

combined_df_long

,Iteration,Type,Exchanges,Value
0,0,Quantity (kg),Product/Tech - Concrete production,2316.000000
1,1,Quantity (kg),Product/Tech - Concrete production,2316.000000
2,2,Quantity (kg),Product/Tech - Concrete production,2316.000000
3,3,Quantity (kg),Product/Tech - Concrete production,2316.000000
4,4,Quantity (kg),Product/Tech - Concrete production,2316.000000
...,...,...,...,...
3995,995,Diff Score,Total - Process Level,15.473938
3996,996,Diff Score,Total - Process Level,101.354832
3997,997,Diff Score,Total - Process Level,-23.955812
3998,998,Diff Score,Total - Process Level,-15.675135


# Export results to excel

In [208]:

name =["00000","11111", "22222", "33333","44444","55555"]


In [209]:

quantities_df["Test"] = uncertainty_Char
MCresults_df["Test"] = uncertainty_Char
combined_df_long["Test"] = uncertainty_Char
quantities_df["Uncertainty Scenario"] = name[uncertainty_Char]
MCresults_df["Uncertainty Scenario"] = name[uncertainty_Char]
combined_df_long["Uncertainty Scenario"] = name[uncertainty_Char]

quantities_df_store = quantities_df
MCresults_df_store = MCresults_df
combined_df_long_store = combined_df_long

%store quantities_df_store
%store MCresults_df_store
%store combined_df_long_store


Stored 'quantities_df_store' (DataFrame)
Stored 'MCresults_df_store' (DataFrame)
Stored 'combined_df_long_store' (DataFrame)


In [210]:
filename='quantitiesMCsimulation_Test_Cement_FixedBio_DQI'+str(uncertainty_Char)+'.xlsx'
excel_file=filename
writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')  


#Export quantities dataframe to excel
quantities_df.to_excel(writer,sheet_name='QuantitiesSample',index=False) # to save it
MCresults_df.to_excel(writer,sheet_name='MCresults',index=False) # to save it
combined_df_long.to_excel(writer,sheet_name='Combined_Long',index=False) # to save it


# save the workbook to a file
writer.save()


print(f"DataFrames exported to {excel_file}")

C:\Users\em398\AppData\Local\Temp\ipykernel_20168\3875295108.py:13: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


DataFrames exported to quantitiesMCsimulation_Test_Cement_FixedBio_DQI3.xlsx
